<a href="https://colab.research.google.com/github/reknahs/email-anonymization/blob/main/azure_anonymization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import email
from email import policy
from email.parser import BytesParser

# Path to your .eml file
eml_file_path = '/content/drive/MyDrive/emails/emailreturn1.eml'
# eml_file_path = '/content/drive/MyDrive/emails/emailreturn2.eml'
# eml_file_path = '/content/drive/MyDrive/emails/emailreturn3.eml'
# eml_file_path = '/content/drive/MyDrive/emails/emailreturn4.eml'

# Read the .eml file
with open(eml_file_path, 'rb') as f:
    msg = BytesParser(policy=policy.default).parse(f)

# Extract subject
subject = msg['subject']
print('Subject:', subject)

# Extract body (both plain text and HTML)
body = ""
if msg.is_multipart():
    for part in msg.iter_parts():
        content_type = part.get_content_type()
        content_disposition = str(part.get('Content-Disposition'))

        # Look for plain text parts
        if content_type == 'text/plain' and 'attachment' not in content_disposition:
            body = part.get_payload(decode=True).decode(part.get_content_charset())
            break
        # Look for HTML parts
        elif content_type == 'text/html' and 'attachment' not in content_disposition:
            body = part.get_payload(decode=True).decode(part.get_content_charset())
            break
else:
    # If the message isn't multipart, the payload is simple text
    body = msg.get_payload(decode=True).decode(msg.get_content_charset())

Subject: Subject: Flip It and Reverse It (Refund On The Way)


In [53]:
%%capture
from google.colab import userdata
userdata.get('API_VERSION')
userdata.get('API_KEY')
userdata.get('AZURE_ENDPOINT')
userdata.get('DEPLOYMENT_NAME')

In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key="c0ad78be7a5d4cd0a50c6a4321b0f593",
    api_version="2024-03-01-preview",
    azure_endpoint = "https://iomd-test-openai.openai.azure.com/"
    )

deployment_name='gpt-35-turbo' #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment.

# Send a completion call to generate an answer
prompt = 'Output this email body after replacing ALL PII with placeholder info: '+body
content = '''
You are an anonymizer of emails. Given an email's body, your job is to output the same email but after replacing ALL Personal Identifiable Information (PII) with placeholder/fake info. PII is any info that permits the identity of an individual to whom the information applies to be reasonably inferred by either direct or indirect means.

You should REPLACE ALL instances of the following for the CUSTOMER ONLY:
-One's name
-One's billing/shipping address
-One's phone number
-One's email address
-One's order number
-One's username
-Any portion of one's credit card number
-One's ip address.

Somethings that should NOT be replaced are:
-The name of the item bought or refunded
-The number of items bought or refunded
-The email, phone number, address, or name of the COMPANY
-The cost of the items

Replace the info with fake names/addresses/phone numbers/emails/order numbers/credit card numbers/etc. Make sure to not remove any info that does not correspond to the person being emailed.
In addition, all links should be deleted entirely.
Here is an example of the converting process you should use for an email:

Input:
Hi James,

We'd like to confirm your order of 5 pairs of shorts (Order number: 6823) to the address 8523 Main St. San Francisco, CA 92356.
We have sent this email to both your email: jamessmith@gmail.com and your phone number 882-235-2362.
Paid with Mastercard ending in 9823.

CONTACT US:
1-235-225-9323
235 2nd St. San Francisco, CA 92356

Example of your output:
Hi Stephen,

We'd like to confirm your order of 5 pairs of shorts (Order number: 9866) to the address 1323 Oak St. Orlando, FL 12235.
We have sent this email to both your email: stephencur2@gmail.com and your phone number 122-003-2123.
Paid with Mastercard ending in 0912.

CONTACT US:
1-235-225-9323
235 2nd St. San Francisco, CA 92356
'''
response = client.chat.completions.create(
    model=deployment_name,
    messages=[{"role": "system", "content": content},
              {"role": "user", "content": prompt}
              ]
)
print(response.choices[0].message)

ChatCompletionMessage(content="Hi John,\n\nWe've refunded your order for $110.00 USD. Heads up: while the refund is immediate on our end, it can take a few days for your bank to process.\nOrder 4658941864\n1 Women's Wool Runners - Dapple Grey (Cream Sole) - 9\t$110.00\nSubtotal\t$110.00\nFast Shipping (Delivered in 3-5 Business Days)\t$6.00\nTotal\t$116.00\n(Paid with Visa...9384)\t$116.00\nRefund to Visa...9384\t- $110.00\n\t\t\nHELP\tCONTACT US\t1.800.123.4567\n\t\t\t\n© 2023 Allbirds, Inc.\n730 Montgomery St. San Francisco, CA 94111", role='assistant', function_call=None, tool_calls=None)


In [ ]:
print(response.choices[0].message.content)

Hi John,

We've refunded your order for $110.00 USD. Heads up: while the refund is immediate on our end, it can take a few days for your bank to process.
Order 4658941864
1 Women's Wool Runners - Dapple Grey (Cream Sole) - 9	$110.00
Subtotal	$110.00
Fast Shipping (Delivered in 3-5 Business Days)	$6.00
Total	$116.00
(Paid with Visa...9384)	$116.00
Refund to Visa...9384	- $110.00
		
HELP	CONTACT US	1.800.123.4567
			
© 2023 Allbirds, Inc.
730 Montgomery St. San Francisco, CA 94111


In [ ]:
print(body)

Hi Rohan,
We've refunded your order for $110.00 USD. Heads up: while the refund is immediate on our end, it can take a few days for your bank to process.
Order 649594665 <https://www.allbirds.com/11044168/orders/5c3c2ad86013c7f694c91b6e754ca11e/authenticate?key=c8dd6330d211f5f96f23357eab2414e5?utm_source=email-t&amp;utm_medium=email&amp;utm_campaign=orefund> 
 <https://cdn.shopify.com/s/files/1/1104/4168/products/WR3MDPG_SHOE_ANGLE_GLOBAL_MENS_WOOL_RUNNER_DAPPLE_GREY_CREAM_81817d0e-7018-4c5c-9f46-8495e51b6cbd_100x80_crop_left.png?v=1679676212> 	1 Women's Wool Runners - Dapple Grey (Cream Sole) - 9	$110.00
Subtotal	$110.00
Fast Shipping (Delivered in 3-5 Business Days)	$6.00
Total	$116.00
(Paid with Visa...3584)	$116.00
Refund to Visa...3584	- $110.00
		
HELP	CONTACT US	1.888.963.8944
			
© 2023 Allbirds, Inc.
730 Montgomery St. San Francisco, CA 94111


